# Basics

In [34]:
%%file mp1.py

import multiprocessing as mp
import os
from time import sleep

def hello_world():
    print('Hello I am %d' % os.getpid())
    sleep(4)

ps = [mp.Process(target=hello_world) for i in range(5)]
for p in ps:
    p.start()
    print('Started')
    
for p in ps:
    p.join()
    
print('Done!')

Overwriting mp1.py


In [33]:
%%bash 
python ./mp1.py

Started
Started
Started
Started
Started
Done!
Hello I am 872
Hello I am 873
Hello I am 874
Hello I am 875
Hello I am 876


## Queues

In [42]:
%%file mp2.py

import multiprocessing as mp
import os

def take_exp(qin, qout):
    while not qin.empty():
        x, e = qin.get()
        print('%d computed %d^%d' % (os.getpid(), x, e))
        qout.put(x**e)

qin = mp.Queue()
qout = mp.Queue()
ps = [mp.Process(target=take_exp, args=(qin, qout)) for i in range(5)]
for p in ps:
    p.start()
    

for i in range(20):
    qin.put((i, 2))
for p in ps:
    p.join()
y = []
while not qout.empty():
    y.append(qout.get())

print(y)

Overwriting mp2.py


In [43]:
%%bash
python mp2.py

1757 computed 0^2
1757 computed 1^2
1757 computed 2^2
1757 computed 3^2
1757 computed 4^2
1757 computed 5^2
1757 computed 6^2
1757 computed 7^2
1757 computed 8^2
1757 computed 9^2
1757 computed 10^2
1757 computed 11^2
1757 computed 12^2
1757 computed 13^2
1757 computed 14^2
1757 computed 15^2
1757 computed 16^2
1757 computed 17^2
1757 computed 18^2
1757 computed 19^2
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]


## Pool

In [37]:
%%file mp3.py

import multiprocessing as mp
import os

def take_exp(a):
    x, e = a
    print('%d computed %d^%d' % (os.getpid(), x, e))
    return x**e

p = mp.Pool(processes=5)
x = [(x, 2) for x in range(20)]
y = p.map(take_exp, x)
print(y)

Writing mp3.py


## Locks

In [53]:
%%file mp4.py
import multiprocessing as mp
import numpy as np
import h5py as h5
import os

def write_to_hdf(lock, group, params):
    lock.acquire()
    print('%d got lock' % os.getpid())
    data = np.random.rand(params['n'], params['m'])
    f = h5.File(params['path'], 'a')
    if group in f:
        del f[group]
    f[group] = data
    f.close()
    lock.release()
    print('%d released lock' % os.getpid())
    
params = dict(path='data.h5', n=10, m=5)
groups = ['d1', 'd2', 'd3']
lock = mp.Lock()
ps = [mp.Process(target=write_to_hdf, args=(lock, g, params)) for g in groups]
for p in ps:
    p.start()
for p in ps:
    p.join()
print('Done!')

Overwriting mp4.py


In [50]:
%%bash
python mp4.py

Process is interrupted.
